In [1]:
import random
import pandas as pd

In [2]:
df = pd.read_csv('data/adventures-list.csv')
df = df.dropna(subset=['Adventure'])
df['Base Rarity'] = df['Base Rarity'].astype('int')
df['Release'] = df['Release'].astype('int')
df.head()

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
0,Taiga,2,Warrior,Dagger,Ruby,Story,Female,Dikaisia,0
1,Zan,2,Cleric,Blade,Sapphire,Story,Male,Haiping,0
2,Rawn,2,Ranger,Axe,Emerald,Story,Male,Egalus,0
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0
4,Kaede,2,Knight,Staff,Onyx,Story,Female,Fur Etopia,0


In [3]:
RATES = {1: 0.75, 2: 0.2, 3: 0.05}
FEATURED = {2: 0.04, 3: 0.01}

### Free Adventures

In [4]:
adv = df.loc[df['Release'] == 0]
adv['Adventure'].unique().tolist()

['Taiga', 'Zan', 'Rawn', 'Ryujin', 'Kaede', 'Sayori']

### Initial Rolls

In [5]:
def simulate_roll(num, release):
    result = pd.DataFrame()
    for n in range(num):
        res = roll(release)
        if res is not None:
            result = result.append(res)
    return result

def roll(release, pity=False):
    pool = df.loc[df['Release'] <= release]
    die1 = random.random()
    if die1 <= RATES[1]:
        return
    elif die1 <= RATES[1] + RATES[2]:
        die2 = random.random()
        adv = None
        if die2 <= FEATURED[2] / RATES[2]:
            adv = get_featured(2, release, pool)
        if adv is None:
            adv = get_random_adventure(2, pool)
        return adv
    else:
        die2 = random.random()
        if die2 <= FEATURED[3] / RATES[3]:
            adv = get_featured(3, release, pool)
        else:
            adv = get_random_adventure(3, pool)
        return adv
            
def get_random_adventure(rarity, pool):
    return pool.loc[pool['Base Rarity'] == rarity].sample()

def get_featured(rarity, release, pool):
    row = pool.loc[(pool['Base Rarity'] == rarity) & (pool['Release'] == release)]
    if len(row) < 1:
        return None
    elif len(row) > 1:
        row = row.sample()
    return row

In [6]:
roll(1)

In [7]:
simulate_roll(30, 1)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
34,Vive,2,Rogue,Sword,Amber,Permanent,Female,Haiping,1
33,Yawen,2,Ranger,Dagger,Amber,Permanent,Female,Haiping,1
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1
15,Kin,3,Rogue,Blade,Sapphire,Permanent,Female,Haiping,1
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1
25,Atari,3,Mage,Gauntlets,Topaz,Permanent,Male,Egalus,1
29,Docward,2,Cleric,Mace,Onyx,Permanent,Male,Fur Etopia,1
22,Ryza,2,Knight,Gun,Topaz,Permanent,Female,Haiping,1


In [8]:
simulate_roll(100, 1)['Base Rarity'].value_counts()

2    16
3     7
Name: Base Rarity, dtype: int64

In [9]:
simulate_roll(10, 2)

""


In [10]:
simulate_roll(10, 3)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
33,Yawen,2,Ranger,Dagger,Amber,Permanent,Female,Haiping,1
17,Gozen,2,Rogue,Blade,Emerald,Permanent,Male,Haiping,1
14,Gifford,2,Mage,Staff,Sapphire,Permanent,Male,Fur Etopia,1
12,Esable,2,Knight,Bow,Sapphire,Permanent,Female,Haiping,1


In [11]:
simulate_roll(10, 15)

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
11,Hyun,2,Warrior,Gauntlets,Sapphire,Permanent,Female,Dikaisia,1
7,Damian,2,Ranger,Gun,Ruby,Permanent,Male,Egalus,1


In [12]:
def get_roll_average(num_samples, num_rolls, release):
    stars2 = 0
    stars2f = 0
    stars3 = 0
    stars3f = 0
    for n in range(num_samples):
        r = simulate_roll(num_rolls, release)
        stars2 += len(r.loc[(r['Base Rarity'] == 2) & (r['Release'] != release)])
        stars2f += len(r.loc[(r['Base Rarity'] == 2) & (r['Release'] == release)])
        stars3 += len(r.loc[(r['Base Rarity'] == 3) & (r['Release'] != release)])
        stars3f += len(r.loc[(r['Base Rarity'] == 3) & (r['Release'] == release)])
    stars2 = round(stars2/num_samples, 2)
    stars2f = round(stars2f/num_samples, 2)
    stars3 = round(stars3/num_samples, 2)
    stars3f = round(stars3f/num_samples, 2)
    return stars2, stars2f, stars3, stars3f

In [13]:
get_roll_average(100, 100, 2)

(15.36, 4.62, 3.58, 1.51)

In [15]:
def roll_till_feature(num_samples, release):
    rolls = []
    featured = df.loc[(df['Base Rarity'] == 3) & (df['Release'] == release), 'Adventure'].item()
    for n in range(num_samples):
        r = 0
        while True:
            r += 1
            adv = roll(release)
            if adv is not None:
                if adv['Adventure'].item() == featured:
                    rolls.append(r)
                    break
    return rolls

In [16]:
roll_till_feature(10, 2)

[244, 50, 1, 65, 92, 38, 66, 5, 4, 55]

In [17]:
rolls = roll_till_feature(100, 2)
sum(rolls) / len(rolls)

63.52

### Team

In [18]:
team = adv.copy()
team

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
0,Taiga,2,Warrior,Dagger,Ruby,Story,Female,Dikaisia,0
1,Zan,2,Cleric,Blade,Sapphire,Story,Male,Haiping,0
2,Rawn,2,Ranger,Axe,Emerald,Story,Male,Egalus,0
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0
4,Kaede,2,Knight,Staff,Onyx,Story,Female,Fur Etopia,0
5,Sayori,2,Mage,Wand,Amber,Story,Female,Haiping,0


In [19]:
roll_res = simulate_roll(50, 1)
team = team.append(roll_res)
roll_res

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
11,Hyun,2,Warrior,Gauntlets,Sapphire,Permanent,Female,Dikaisia,1
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1
17,Gozen,2,Rogue,Blade,Emerald,Permanent,Male,Haiping,1
27,Neka,2,Rogue,Axe,Onyx,Permanent,Female,Dikaisia,1
5,Sayori,2,Mage,Wand,Amber,Story,Female,Haiping,0
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1
31,Devon,2,Warrior,Mace,Amber,Permanent,Male,Egalus,1
22,Ryza,2,Knight,Gun,Topaz,Permanent,Female,Haiping,1
24,Micah,2,Cleric,Mace,Topaz,Permanent,Male,Fur Etopia,1
25,Atari,3,Mage,Gauntlets,Topaz,Permanent,Male,Egalus,1


In [20]:
roll_res = simulate_roll(20, 2)
team = team.append(roll_res)
roll_res

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1
7,Damian,2,Ranger,Gun,Ruby,Permanent,Male,Egalus,1


In [21]:
roll_res = simulate_roll(10, 3)
team = team.append(roll_res)
roll_res

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
15,Kin,3,Rogue,Blade,Sapphire,Permanent,Female,Haiping,1
17,Gozen,2,Rogue,Blade,Emerald,Permanent,Male,Haiping,1
36,Elsie,2,Mage,Sword,Topaz,Permanent,Female,Fur Etopia,2


In [22]:
roll_res = simulate_roll(10, 4)
team = team.append(roll_res)
roll_res

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
27,Neka,2,Rogue,Axe,Onyx,Permanent,Female,Dikaisia,1
38,Dred,2,Ranger,Lance,Onyx,Permanent,Male,Haiping,3
40,Lingxin,2,Cleric,Dagger,Amber,Permanent,Female,Haiping,4
13,Offenbach,2,Ranger,Dagger,Sapphire,Permanent,Male,Egalus,1


In [23]:
roll_res = simulate_roll(10, 5)
team = team.append(roll_res)
roll_res

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release
42,Lionel,2,Knight,Mace,Ruby,Permanent,Male,Fur Etopia,5
8,Senzo,2,Mage,Lance,Ruby,Permanent,Male,Dikaisia,1


In [24]:
team['Count'] = team.groupby('Adventure')['Release'].transform('count')
team = team.drop_duplicates()
team

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
0,Taiga,2,Warrior,Dagger,Ruby,Story,Female,Dikaisia,0,1
1,Zan,2,Cleric,Blade,Sapphire,Story,Male,Haiping,0,1
2,Rawn,2,Ranger,Axe,Emerald,Story,Male,Egalus,0,1
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0,1
4,Kaede,2,Knight,Staff,Onyx,Story,Female,Fur Etopia,0,1
5,Sayori,2,Mage,Wand,Amber,Story,Female,Haiping,0,2
11,Hyun,2,Warrior,Gauntlets,Sapphire,Permanent,Female,Dikaisia,1,1
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1,3
17,Gozen,2,Rogue,Blade,Emerald,Permanent,Male,Haiping,1,2
27,Neka,2,Rogue,Axe,Onyx,Permanent,Female,Dikaisia,1,2


In [25]:
team.loc[team['Element'] == 'Ruby']

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
0,Taiga,2,Warrior,Dagger,Ruby,Story,Female,Dikaisia,0,1
9,Panlei,2,Cleric,Wand,Ruby,Permanent,Female,Haiping,1,1
7,Damian,2,Ranger,Gun,Ruby,Permanent,Male,Egalus,1,1
42,Lionel,2,Knight,Mace,Ruby,Permanent,Male,Fur Etopia,5,1
8,Senzo,2,Mage,Lance,Ruby,Permanent,Male,Dikaisia,1,1


In [26]:
team.loc[team['Element'] == 'Sapphire']

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
1,Zan,2,Cleric,Blade,Sapphire,Story,Male,Haiping,0,1
11,Hyun,2,Warrior,Gauntlets,Sapphire,Permanent,Female,Dikaisia,1,1
12,Esable,2,Knight,Bow,Sapphire,Permanent,Female,Haiping,1,1
15,Kin,3,Rogue,Blade,Sapphire,Permanent,Female,Haiping,1,1
13,Offenbach,2,Ranger,Dagger,Sapphire,Permanent,Male,Egalus,1,1


In [27]:
team.loc[team['Element'] == 'Emerald']

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
2,Rawn,2,Ranger,Axe,Emerald,Story,Male,Egalus,0,1
17,Gozen,2,Rogue,Blade,Emerald,Permanent,Male,Haiping,1,2


In [28]:
team.loc[team['Element'] == 'Topaz']

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
3,Ryujin,2,Rogue,Lance,Topaz,Story,Male,Dikaisia,0,1
22,Ryza,2,Knight,Gun,Topaz,Permanent,Female,Haiping,1,1
24,Micah,2,Cleric,Mace,Topaz,Permanent,Male,Fur Etopia,1,1
25,Atari,3,Mage,Gauntlets,Topaz,Permanent,Male,Egalus,1,1
36,Elsie,2,Mage,Sword,Topaz,Permanent,Female,Fur Etopia,2,1


In [29]:
team.loc[team['Element'] == 'Onyx']

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
4,Kaede,2,Knight,Staff,Onyx,Story,Female,Fur Etopia,0,1
27,Neka,2,Rogue,Axe,Onyx,Permanent,Female,Dikaisia,1,2
38,Dred,2,Ranger,Lance,Onyx,Permanent,Male,Haiping,3,1


In [30]:
team.loc[team['Element'] == 'Amber']

,Adventure,Base Rarity,Class,Weapon,Element,Availability,Gender,Affiliation,Release,Count
5,Sayori,2,Mage,Wand,Amber,Story,Female,Haiping,0,2
35,Miki,3,Cleric,Gun,Amber,Permanent,Female,Dikaisia,1,3
31,Devon,2,Warrior,Mace,Amber,Permanent,Male,Egalus,1,1
40,Lingxin,2,Cleric,Dagger,Amber,Permanent,Female,Haiping,4,1
